In [ ]:
# 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 경로 이동
%cd /content/drive/MyDrive/dataset

/content/drive/MyDrive/dataset


In [ ]:
# 파일 리스트 확인
%ls

# https://www.kaggle.com/datasets/chaitanyakakade77/american-sign-language-dataset 데이터 출처

archive.zip  ASL_Data/  hand_landmarker.task


In [ ]:
# 파일 압축 풀기
!unzip -qq "/content/drive/MyDrive/dataset/archive.zip"
# !unzip "New_sample.zip"

In [ ]:
# 압축 풀기 확인

import os

def count_files_in_subfolders(root_folder):
    """
    주어진 루트 폴더 아래의 모든 하위 폴더를 돌면서 각 폴더의 파일 수를 출력합니다.

    Args:
        root_folder: 파일 수를 셀 루트 폴더의 경로.
    """
    for foldername, subfolders, filenames in os.walk(root_folder):
        file_count = len(filenames)
        print(f"폴더: {foldername}, 파일 수: {file_count}")

# 예시: 'ASL_Data' 폴더 아래의 파일 수를 세려면
root_path = 'ASL_Data'
count_files_in_subfolders(root_path)


def count_specific_files_in_subfolders(root_folder, extensions):
    """
    주어진 루트 폴더 아래의 모든 하위 폴더를 돌면서 특정 확장자를 가진 파일 수를 출력합니다.

    Args:
        root_folder: 파일 수를 셀 루트 폴더의 경로.
        extensions: 셀 파일 확장자 목록 (예: ['.jpg', '.png']).
    """
    for foldername, subfolders, filenames in os.walk(root_folder):
        file_count = 0
        for filename in filenames:
            if any(filename.lower().endswith(ext) for ext in extensions):
                file_count += 1
        print(f"폴더: {foldername}, {extensions} 파일 수: {file_count}")

# 예시: 'ASL_Data' 폴더 아래의 jpg, png 파일 수를 세려면
root_path = 'ASL_Data'
extensions_to_count = ['.jpg', '.jpeg', '.png']
count_specific_files_in_subfolders(root_path, extensions_to_count)

폴더: ASL_Data, 파일 수: 0
폴더: ASL_Data/A-samples, 파일 수: 1004
폴더: ASL_Data/B-samples, 파일 수: 1001
폴더: ASL_Data/C-samples, 파일 수: 1012
폴더: ASL_Data/D-samples, 파일 수: 1008
폴더: ASL_Data/E-samples, 파일 수: 1019
폴더: ASL_Data/F-samples, 파일 수: 1010
폴더: ASL_Data/G-samples, 파일 수: 1010
폴더: ASL_Data/H-samples, 파일 수: 1012
폴더: ASL_Data/I-samples, 파일 수: 1010
폴더: ASL_Data/J-samples, 파일 수: 1010
폴더: ASL_Data/K-samples, 파일 수: 1015
폴더: ASL_Data/L-samples, 파일 수: 1008
폴더: ASL_Data/M-samples, 파일 수: 1013
폴더: ASL_Data/N-samples, 파일 수: 1013
폴더: ASL_Data/O-samples, 파일 수: 1017
폴더: ASL_Data/P-samples, 파일 수: 1008
폴더: ASL_Data/Q-samples, 파일 수: 1012
폴더: ASL_Data/R-samples, 파일 수: 1010
폴더: ASL_Data/S-samples, 파일 수: 1009
폴더: ASL_Data/T-samples, 파일 수: 1011
폴더: ASL_Data/U-samples, 파일 수: 1012
폴더: ASL_Data/V-samples, 파일 수: 1014
폴더: ASL_Data/W-samples, 파일 수: 1014
폴더: ASL_Data/X-samples, 파일 수: 1010
폴더: ASL_Data/Y-samples, 파일 수: 1016
폴더: ASL_Data/Z-samples, 파일 수: 1053
폴더: ASL_Data, ['.jpg', '.jpeg', '.png'] 파일 수: 0
폴더: ASL_Data/A-sampl

In [ ]:
!pip install -q mediapipe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 52.4 MB/s eta 0:00:00


In [ ]:
!wget -q https://storage.googleapis.com/mediapipe-models/hand_landmarker/hand_landmarker/float16/1/hand_landmarker.task

In [ ]:
#@markdown We implemented some functions to visualize the hand landmark detection results. <br/> Run the following cell to activate the functions.

from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np

MARGIN = 10  # pixels
FONT_SIZE = 1
FONT_THICKNESS = 1
HANDEDNESS_TEXT_COLOR = (88, 205, 54) # vibrant green

def draw_landmarks_on_image(rgb_image, detection_result):
  hand_landmarks_list = detection_result.hand_landmarks
  handedness_list = detection_result.handedness
  annotated_image = np.copy(rgb_image)

  # Loop through the detected hands to visualize.
  for idx in range(len(hand_landmarks_list)):
    hand_landmarks = hand_landmarks_list[idx]
    handedness = handedness_list[idx]

    # Draw the hand landmarks.
    hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    hand_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      hand_landmarks_proto,
      solutions.hands.HAND_CONNECTIONS,
      solutions.drawing_styles.get_default_hand_landmarks_style(),
      solutions.drawing_styles.get_default_hand_connections_style())

    # Get the top left corner of the detected hand's bounding box.
    height, width, _ = annotated_image.shape
    x_coordinates = [landmark.x for landmark in hand_landmarks]
    y_coordinates = [landmark.y for landmark in hand_landmarks]
    text_x = int(min(x_coordinates) * width)
    text_y = int(min(y_coordinates) * height) - MARGIN

    # Draw handedness (left or right hand) on the image.
    cv2.putText(annotated_image, f"{handedness[0].category_name}",
                (text_x, text_y), cv2.FONT_HERSHEY_DUPLEX,
                FONT_SIZE, HANDEDNESS_TEXT_COLOR, FONT_THICKNESS, cv2.LINE_AA)

  return annotated_image

In [ ]:
import pandas as pd

def convert_landmarks_to_dataframe(detection_result, folder_name):
    """
    MediaPipe의 손 랜드마크 감지 결과를 판다스 데이터프레임으로 변환합니다.

    Args:
        detection_result: MediaPipe HandLandmarker의 감지 결과

    Returns:
        pandas.DataFrame: 각 손가락 랜드마크의 x, y, z 좌표가 포함된 데이터프레임
    """
    all_hands_data = []

    for hand_idx, hand_landmarks in enumerate(detection_result.hand_landmarks):
        hand_data = []

        # 각 랜드마크의 x, y, z 좌표를 추출
        for idx, landmark in enumerate(hand_landmarks):
            hand_data.extend([
                landmark.x,
                landmark.y,
                landmark.z
            ])

        # 손의 번호와 좌표 데이터를 딕셔너리로 변환
        hand_dict = {
            'hand_index': hand_idx,
            **{f'landmark_{i}_{coord}': val
               for i in range(21)  # MediaPipe는 각 손마다 21개의 랜드마크를 감지
               for coord, val in zip(['x', 'y', 'z'], hand_data[i*3:(i+1)*3])}
        }
        all_hands_data.append(hand_dict)

    # 데이터프레임 생성
    df = pd.DataFrame(all_hands_data)
    df['result'] = folder_name[0]
    return df

In [ ]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import os
import pandas as pd


# MediaPipe HandLandmarker 초기화
base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')
options = vision.HandLandmarkerOptions(base_options=base_options, num_hands=2)
detector = vision.HandLandmarker.create_from_options(options)

# ASL_Data 폴더 경로
asl_data_path = 'ASL_Data'

# 모든 랜드마크 데이터를 저장할 리스트
all_landmarks = []

# A-Z 폴더 순회
for folder_name in sorted(os.listdir(asl_data_path)):
    folder_path = os.path.join(asl_data_path, folder_name)
    if os.path.isdir(folder_path):
        print(f"Processing folder: {folder_name}")
        file_list = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))] # 파일 리스트 생성
        total_files = len(file_list) # 폴더 내 총 파일 수
        for index, filename in enumerate(file_list): # enumerate 사용
            image_path = os.path.join(folder_path, filename)
            print(f"  Processing file: {folder_name} - {filename} ({index + 1}/{total_files})") # 진행도 출력

            try:
                image = mp.Image.create_from_file(image_path)
                detection_result = detector.detect(image)

                landmarks_df = convert_landmarks_to_dataframe(detection_result, folder_name)
                if not landmarks_df.empty:
                    all_landmarks.append(landmarks_df)

            except Exception as e:
                print(f"    Error processing {image_path}: {e}")

# 모든 랜드마크 데이터를 하나의 DataFrame으로 병합
if all_landmarks:
    final_df = pd.concat(all_landmarks, ignore_index=True)

    # CSV 파일로 저장
    final_df.to_csv('all_landmarks.csv', index=False)
    print("랜드마크 데이터가 all_landmarks.csv 파일에 저장되었습니다.")
else:
    print("처리할 이미지 또는 랜드마크 데이터가 없습니다.")

# detector 닫기
detector.close()

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  Processing file: V-samples - 187.jpg (113/1014)
  Processing file: V-samples - 188.jpg (114/1014)
  Processing file: V-samples - 189.jpg (115/1014)
  Processing file: V-samples - 19.jpg (116/1014)
  Processing file: V-samples - 190.jpg (117/1014)
  Processing file: V-samples - 191.jpg (118/1014)
  Processing file: V-samples - 192.jpg (119/1014)
  Processing file: V-samples - 193.jpg (120/1014)
  Processing file: V-samples - 194.jpg (121/1014)
  Processing file: V-samples - 195.jpg (122/1014)
  Processing file: V-samples - 196.jpg (123/1014)
  Processing file: V-samples - 197.jpg (124/1014)
  Processing file: V-samples - 198.jpg (125/1014)
  Processing file: V-samples - 199.jpg (126/1014)
  Processing file: V-samples - 2.jpg (127/1014)
  Processing file: V-samples - 20.jpg (128/1014)
  Processing file: V-samples - 200.jpg (129/1014)
  Processing file: V-samples - 201.jpg (130/1014)
  Processing file: V-samples - 202.jpg (131/1014)
  Processing file:

In [ ]:
# 데이터 로드
data = pd.read_csv('all_landmarks.csv')

In [ ]:
# 불필요 열 제거
data.drop('hand_index', axis=1, inplace=True)

In [ ]:
print(data.head())

   landmark_0_x  landmark_0_y  landmark_0_z  landmark_1_x  landmark_1_y  \
0      0.443537      0.837238     -0.000001      0.307017      0.731058   
1      0.445761      0.837874     -0.000001      0.306444      0.732813   
2      0.446503      0.850963     -0.000001      0.318634      0.723006   
3      0.493054      0.792986     -0.000001      0.373296      0.666247   
4      0.433177      0.821334     -0.000001      0.303584      0.710813   

   landmark_1_z  landmark_2_x  landmark_2_y  landmark_2_z  landmark_3_x  ...  \
0     -0.044804      0.187404      0.574844     -0.071507      0.146549  ...   
1     -0.045281      0.187341      0.574983     -0.071442      0.151493  ...   
2     -0.041696      0.226048      0.549309     -0.065365      0.216498  ...   
3     -0.039437      0.285332      0.503460     -0.064564      0.272421  ...   
4     -0.039511      0.205084      0.552309     -0.064400      0.181897  ...   

   landmark_18_x  landmark_18_y  landmark_18_z  landmark_19_x  landm

In [ ]:
# 결측치 확인

# 모든 열과 행 표시
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print(data.isnull().sum())

landmark_0_x     0
landmark_0_y     0
landmark_0_z     0
landmark_1_x     0
landmark_1_y     0
landmark_1_z     0
landmark_2_x     0
landmark_2_y     0
landmark_2_z     0
landmark_3_x     0
landmark_3_y     0
landmark_3_z     0
landmark_4_x     0
landmark_4_y     0
landmark_4_z     0
landmark_5_x     0
landmark_5_y     0
landmark_5_z     0
landmark_6_x     0
landmark_6_y     0
landmark_6_z     0
landmark_7_x     0
landmark_7_y     0
landmark_7_z     0
landmark_8_x     0
landmark_8_y     0
landmark_8_z     0
landmark_9_x     0
landmark_9_y     0
landmark_9_z     0
landmark_10_x    0
landmark_10_y    0
landmark_10_z    0
landmark_11_x    0
landmark_11_y    0
landmark_11_z    0
landmark_12_x    0
landmark_12_y    0
landmark_12_z    0
landmark_13_x    0
landmark_13_y    0
landmark_13_z    0
landmark_14_x    0
landmark_14_y    0
landmark_14_z    0
landmark_15_x    0
landmark_15_y    0
landmark_15_z    0
landmark_16_x    0
landmark_16_y    0
landmark_16_z    0
landmark_17_x    0
landmark_17_

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder


# 특징과 레이블 분리
X = data.drop('result', axis=1)
y = data['result']

# 레이블 인코딩 (Label Encoding)
le = LabelEncoder()
y = le.fit_transform(y) # y = pd.get_dummies(y) One Hot Encoding

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 생성 및 학습
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# 예측
y_pred = model.predict(X_test)

# 평가
print(classification_report(y_test, y_pred))




              precision    recall  f1-score   support

           0       1.00      1.00      1.00       190
           1       1.00      1.00      1.00       184
           2       1.00      1.00      1.00       226
           3       1.00      1.00      1.00       218
           4       1.00      1.00      1.00       200
           5       1.00      1.00      1.00       198
           6       1.00      1.00      1.00       221
           7       1.00      1.00      1.00       214
           8       1.00      1.00      1.00       218
           9       1.00      1.00      1.00       207
          10       1.00      1.00      1.00       201
          11       1.00      1.00      1.00       193
          12       1.00      0.99      0.99       208
          13       0.98      0.99      0.99       186
          14       1.00      1.00      1.00       203
          15       1.00      1.00      1.00       194
          16       1.00      1.00      1.00       181
          17       1.00    

In [ ]:
# 모델 저장
import joblib

joblib.dump(model, 'sign_language_model.joblib')

['sign_language_model.joblib']